# 记忆功能
SimpleMemory 是 LangChain 中最基础的记忆组件之一，它可以存储和检索键值对形式的信息。

In [12]:
import os
from langchain.memory import SimpleMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# 创建一个 SimpleMemory 实例
memory = SimpleMemory(memories={
    "person_name": "Alice",
    "favorite_color": "blue"
})

# 创建一个提示模板
template = """
根据以下信息回答问题:
人名: {person_name}
最喜欢的颜色: {favorite_color}

问题: {question}
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

# 创建 LLM
llm = ChatOpenAI(
    model = "deepseek-r1",
    api_key = os.environ["API_KEY"],
    base_url=os.environ["API_BASEURL"]
)

# 创建一个函数来合并 memory 和输入
def merge_memory(input_dict):
    return {
        **input_dict,
        **memory.memories
    }

# 创建 Chain，并将 memory 传入
chain = (
RunnablePassthrough()|
merge_memory|
prompt|
llm
)

# 使用 chain
response = chain.invoke({"question": "这个人最喜欢什么颜色？"})
print(response)

content='根据提供的信息，Alice最喜欢的颜色是蓝色（blue）。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 217, 'prompt_tokens': 26, 'total_tokens': 243, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-r1', 'system_fingerprint': None, 'id': 'chatcmpl-a48b0037-5fc5-9cde-b069-c000c2714b0d', 'finish_reason': 'stop', 'logprobs': None} id='run-b66e7d91-bbc6-45cd-a45a-e5af38fb538a-0' usage_metadata={'input_tokens': 26, 'output_tokens': 217, 'total_tokens': 243, 'input_token_details': {}, 'output_token_details': {}}
